Importing Necessary Libraries

In [3]:
import os
import zipfile
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import random
from PIL import Image
import numpy as np
import os
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from datetime import datetime
from keras.optimizers import SGD

In [20]:
def create_data_directory(datasetName, classnames = []):
  for classname in classnames:
    trainDir = os.path.join('data',datasetName,'train',classname)
    testDir = os.path.join('data',datasetName,'test',classname)

    if not os.path.exists(trainDir):
      os.makedirs(trainDir)

    if not os.path.exists(testDir):
      os.makedirs(testDir)

In [21]:
def saveNumpyImage(imageArray, label, image_name_suffix, dataSetName, trainOrTest):
  if trainOrTest not in ["train", "test"]:
    print("The parameter trainOrTest value should be either 'train' or 'test'")
    return False

  imageDirectory = os.path.join('data',dataSetName,trainOrTest,label)

  if not os.path.exists(imageDirectory):
    print("The image directory does not exists.{}".format(imageDirectory))
    return False

  im = Image.fromarray(imageArray)
  im.save(os.path.join(imageDirectory,label+"_"+image_name_suffix+".jpeg"))

  return True

Loading the MNIST Dataset. The dataset has 10 classes. The dataset contains images of 10 digits from 0 to 9.

In [4]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

In [40]:
create_data_directory("MNIST",["0","1","2","3","4","5","6","7","8","9"])

In [42]:
for i,image in enumerate(x_train):
  saveNumpyImage(image,str(y_train[i]),str(i),"MNIST", "train")

In [43]:
for i,image in enumerate(x_test):
  saveNumpyImage(image,str(y_test[i]),str(i),"MNIST", "test")

In [4]:
def train_val_generators(TRAINING_DIR, VALIDATION_DIR):
  train_datagen = ImageDataGenerator(rescale=1./255,
                                     rotation_range=40,
                                     width_shift_range=0.2,
                                     height_shift_range=0.2,
                                     shear_range=0.2,
                                     zoom_range=0.2,
                                     horizontal_flip=True,
                                     fill_mode='nearest')

  train_generator = train_datagen.flow_from_directory(directory=TRAINING_DIR,
                                                      batch_size=32,
                                                      class_mode='categorical',
                                                      target_size=(227, 227))

  validation_datagen = ImageDataGenerator(rescale=1./255)

  validation_generator = validation_datagen.flow_from_directory(directory=VALIDATION_DIR,
                                                                batch_size=32,
                                                                class_mode='categorical',
                                                                target_size=(227, 227))

  return train_generator, validation_generator

In [5]:
train_dir = os.path.join("data","MNIST","train")
validation_dir = os.path.join("data","MNISt","test")

In [6]:
train_generator, validation_generator = train_val_generators(train_dir, validation_dir)

Found 60000 images belonging to 10 classes.
Found 10000 images belonging to 10 classes.


In [25]:
def create_model():
  model = tf.keras.models.Sequential([
      tf.keras.layers.Conv2D(96, (11,11), strides=4, activation='relu', kernel_initializer= 'he_normal', padding= 'valid',input_shape=(227,227,1)),
      tf.keras.layers.MaxPool2D(pool_size=(3,3), strides= (2,2), padding= 'valid', data_format= None ),

      tf.keras.layers.Conv2D(256, (5,5), strides=1, activation='relu', kernel_initializer= 'he_normal', padding= 'same'),
      tf.keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2), padding= 'valid', data_format= None ),

      tf.keras.layers.Conv2D(384, (3,3), strides=1, activation='relu', kernel_initializer= 'he_normal', padding='same'),
      tf.keras.layers.Conv2D(384, (3,3), strides=1, activation='relu', kernel_initializer= 'he_normal', padding='same'),
      tf.keras.layers.Conv2D(256, (3,3), strides=1, activation='relu', kernel_initializer= 'he_normal', padding='same'),
      tf.keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2), padding= 'valid', data_format= None ),

      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(4096, activation='relu'),
      tf.keras.layers.Dense(4096, activation='relu'),
      tf.keras.layers.Dense(1000, activation='relu'),
      tf.keras.layers.Dense(10, activation='softmax')
  ])


  model.compile(optimizer=SGD(learning_rate=0.001),
                loss='categorical_crossentropy',
                metrics=['accuracy', 'AUC', 'FalseNegatives', 'FalsePositives', 'Precision', 'Recall', 'TrueNegatives','TruePositives'])

  return model

In [26]:
model = create_model()

history = model.fit(train_generator,
                    epochs=15,
                    verbose=1,
                    validation_data=validation_generator)

Epoch 1/15
1875/1875 [==============================] - 923s 491ms/step - loss: 1.2658 - accuracy: 0.5698 - auc: 0.9065 - false_negatives: 37242.0000 - false_positives: 5279.0000 - precision: 0.8117 - recall: 0.3793 - true_negatives: 534721.0000 - true_positives: 22758.0000 - val_loss: 0.4404 - val_accuracy: 0.8562 - val_auc: 0.9898 - val_false_negatives: 1997.0000 - val_false_positives: 762.0000 - val_precision: 0.9131 - val_recall: 0.8003 - val_true_negatives: 89238.0000 - val_true_positives: 8003.0000
Epoch 2/15
1875/1875 [==============================] - 788s 420ms/step - loss: 0.6393 - accuracy: 0.7905 - auc: 0.9757 - false_negatives: 16875.0000 - false_positives: 6750.0000 - precision: 0.8647 - recall: 0.7188 - true_negatives: 533250.0000 - true_positives: 43125.0000 - val_loss: 0.2742 - val_accuracy: 0.9143 - val_auc: 0.9954 - val_false_negatives: 1124.0000 - val_false_positives: 524.0000 - val_precision: 0.9443 - val_recall: 0.8876 - val_true_negatives: 89476.0000 - val_true_p

In [34]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [23]:
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

NameError: name 'PhysicalDevice' is not defined

In [27]:
model.save("mnist_alexnet.h5")

In [7]:
import time
loaded_model = tf.keras.models.load_model("mnist_alexnet.h5")
start = time.time()
loaded_model.evaluate(validation_generator, verbose=1)
stop = time.time()
print(f'Inferencing time: {(stop-start)/60} minutes')

313/313 [==============================] - 37s 73ms/step - loss: 0.0644 - accuracy: 0.9778 - auc: 0.9995 - false_negatives: 254.0000 - false_positives: 192.0000 - precision: 0.9807 - recall: 0.9746 - true_negatives: 89808.0000 - true_positives: 9746.0000
Inferencing time: 0.6299274643262227 minutes
